In [116]:
#import datasets
from aif360.datasets import StandardDataset
from aif360.datasets import CompasDataset
#import fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, confusion_matrix

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt


In [123]:
#load COMPAS dataset

try:
    compas = CompasDataset(
        protected_attribute_names=['sex', 'race'],
        privileged_classes=[['Female'], ['Caucasian']], 
        features_to_keep=['age', 'c_charge_degree', 'race', 'age_cat', 
                          'sex', 'priors_count', 'days_b_screening_arrest', 'c_charge_desc'],
        features_to_drop=[],
        categorical_features=['age_cat', 'c_charge_degree', 'c_charge_desc'],
        label_name='two_year_recid'
    )
    print("Dataset loaded successfully!")

    #returns the dataframe and the metadata in a tuple
    df, meta = compas.convert_to_dataframe()

except Exception as e:
    print(f"Error loading COMPAS dataset: {e}")

Dataset loaded successfully!


In [124]:
# copy dataset to ensure original remains unchanged
df = df.copy()

#separate features and labels
features = ['race', 'sex', 'priors_count', 'c_charge_degree=F', 'c_charge_degree=M']
target = 'two_year_recid' #binary target where 0 means does not offend, 1 means offends

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

X_test_indices = X_test.index

#retrive each instance's group membership before scaling X_test to make predictions 
#scaling will make it lose the index information to retrieve this information
grp_membership = df.loc[X_test_indices, 'race'].values
print("\n group membership: ", grp_membership, "\n")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(solver='lbfgs')
model.fit(X_train_scaled, y_train)

#predicted class labels 0 or 1
y_pred = model.predict(X_test_scaled)


 group membership:  [1. 1. 0. ... 0. 0. 0.] 



#### Task 1: Three input arrays for new_metric function

To do: Have three arrays each for predictions, ground truth labels, and group membership from the dataset and model.

Important aspect of this part: indices of each array needs to align such that pred[0] refers to ground_truth[0] and grp_membership[0]. 

The arrays:
1. predictions: positive and negative predictions from the classifier.
2. ground_truth: this is the two_yr_recid column that represents the target variables. So, use y_test which contains ground truth values from the train_test_split for the dataset.
3. grp_membership: array of group membership for each instance containing privileged (label 1 - Caucasian) and non-privileged (label 0 - not Caucasian) as defined by protected attribute of 'race'.


In [125]:
#predictions array
predictions = y_pred 
print("predictions: ", predictions)

#ground truth labels array
ground_truth = y_test.values 
print("ground truth labels: ", ground_truth)

#group membership array defined above where model is trained.
print("group membership: ", grp_membership)

predictions:  [0. 0. 0. ... 0. 0. 0.]
ground truth labels:  [0. 0. 1. ... 0. 1. 1.]
group membership:  [1. 1. 0. ... 0. 0. 0.]


#### Task 2: Coding the new metric using the three arrays as input.

In [145]:
def new_metric(arr_pred, arr_true, arr_grp): #varying these three for diff distri- protected groups can be diff sizes, edge cases 50 TP/50 TN, 10/90
    #Two arrays for privileged and not privileged 
    #g1 and g2- contain predictions and trues are lists of lists [[], []]
    grp_priv = [[], []]
    grp_unpriv = [[], []]

    #j is the number of unique groups in arr_grp - an implicit parameter.
    j = len(set(arr_grp))
    #print("total number of unique groups: ", j)

    for i, label in enumerate(arr_grp):
        #for privileged class
        if label == 1.0:
            #add the corresponding prediction + gt label for that class using the index associated with that label
            grp_priv[0].append(arr_pred[i])
            grp_priv[1].append(arr_true[i])
        
        #for unprivileged class 
        else:
            grp_unpriv[0].append(arr_pred[i])
            grp_unpriv[1].append(arr_true[i])
    
    #print("Privileged group: ", grp_priv)
    #print("Unprivileged group: ", grp_unpriv)
    
    priv_indiv_bi = [] #stores individual benefit value of each instance
    priv_grp_bi = 0 #tracks total benefit for group
    
    #1. for each index in a group calculate the benefit, bi
    for pred, gt in zip(grp_priv[0], grp_priv[1]):
        #calculate benefit for each instance in a group with new range, 0,1,-1 
        #we remove '+1' to add variance? -- need to clarify how to verbalise this specific part
        indiv_benefit = (int(pred) - int(gt)) 
        
        #2. Sum the total benefit of each group
        priv_grp_bi += indiv_benefit
        
        #3. divide by size of group which is total number of instances in each group.
        priv_av_bi = priv_grp_bi / len(grp_priv[0]) #[0] has predictions which will give that number.

        #store individual benefit of each instance in a list
        priv_indiv_bi.append(indiv_benefit)
        
    #print(priv_indiv_bi)
    #print(priv_grp_bi)
    #print(priv_av_bi)
    

    unpriv_indiv_bi = []
    unpriv_grp_bi = 0
    for pred, gt in zip(grp_unpriv[0], grp_unpriv[1]):
        indiv_benefit = (int(pred) - int(gt))  
        unpriv_grp_bi += indiv_benefit
        unpriv_av_bi = unpriv_grp_bi / len(grp_priv[0])
        unpriv_indiv_bi.append(indiv_benefit)

    #print(unpriv_indiv_bi)
    #print(unpriv_grp_bi)
    #print(unpriv_av_bi)
   

    #4. division result is divided by the sum of g1 and g2 - J
    result = (priv_av_bi + unpriv_av_bi) / j

    #print("new metric value: ", result)
    return result, priv_av_bi, unpriv_av_bi

new_metric(predictions, ground_truth, grp_membership)


(-0.26034063260340634, -0.26277372262773724, -0.25790754257907544)